## Bit String Comparison using Quantum Superposition
    

In [1]:
# import libraries
import sys
import numpy as np
import math
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import execute

If we can use quantum states to represent genetic codes, we may be able to compare them, and/or find similar genetic codes quickly.

For example, according to this site the starts of the genetic codes for the Yeast Mitochondrial, Protozoan Mitochondrial, and Bacterial Code are respectively as follow.

Notice that each of the codes is represented by a bitstring of length 64. By comparing characters at the same position in the strings, we can see that Protozoan's is closer to Bacterial's than Yeast's.

Exploiting quantum superposition, we can create quantum states by using only 7 qubits such that each of the quantum states corresponds to the genetic code of Yeast, Protozoan, and Bacterial. We then compare the closeness of their genetic codes by comparing their quantum states, which is made possible by the reversibility of quantum circuit.

Let say we have three genetic codes as below.

In [2]:
YEAST     = "----------------------------------MM----------------------------"
PROTOZOAN = "--MM---------------M------------MMMM---------------M------------"
BACTERIAL = "---M---------------M------------MMMM---------------M------------"


In [3]:
# this function encodes the given string into quantum bits

def encode_bitstring(bitstring, qr, cr, inverse=False):
    """
    create a circuit for constructing the quantum superposition of the bitstring
    """
    n = math.ceil(math.log2(len(bitstring))) + 1                 #number of qubits
    assert n > 2, "the length of bitstring must be at least 2"
    
    qc = QuantumCircuit(qr, cr)
    
    #the probability amplitude of the desired state
    desired_vector = np.array([ 0.0 for i in range(2**n) ])     #initialize to zero
    amplitude = np.sqrt(1.0/2**(n-1))
    
    for i, b in enumerate(bitstring):
        pos = i * 2
        if b == "1" or b == "M":
            pos += 1
        desired_vector[pos] = amplitude
    if not inverse:
        qc.initialize(desired_vector, [ qr[i] for i in range(n) ] )
        qc.barrier(qr)
    else:
        qc.initialize(desired_vector, [ qr[i] for i in range(n) ] ).inverse()  #invert the circuit
        for i in range(n):
            qc.measure(qr[i], cr[i])
    print()
    return qc

In [4]:
# create quantum circuits to create quantum states for Yeast's, Protozoa's and Bacterail's

n = math.ceil(math.log2(len(YEAST))) + 1                 #number of qubits
qr = QuantumRegister(n)
cr = ClassicalRegister(n)

qc_yeast     = encode_bitstring(YEAST, qr, cr)
qc_protozoan = encode_bitstring(PROTOZOAN, qr, cr)
qc_bacterial = encode_bitstring(BACTERIAL, qr, cr)

circs = {"YEAST": qc_yeast, "PROTOZOAN": qc_protozoan, "BACTERIAL": qc_bacterial}


In [5]:
# now we invert the quantum circuits by inverse() function

inverse_qc_yeast     = encode_bitstring(YEAST,     qr, cr, inverse=True)
inverse_qc_protozoan = encode_bitstring(PROTOZOAN, qr, cr, inverse=True)
inverse_qc_bacterial = encode_bitstring(BACTERIAL, qr, cr, inverse=True)

inverse_circs = {"YEAST": inverse_qc_yeast, "PROTOZOAN": inverse_qc_protozoan, "BACTERIAL": inverse_qc_bacterial}

In [6]:
# We can now compare how close the starts of the genetic codes of Protozoan to Yeast's and Bacterial's by performing the test.

from qiskit import IBMQ, BasicAer

key = "PROTOZOAN"       #the name of the code used as key to find similar ones

# use local simulator
backend = BasicAer.get_backend("qasm_simulator")
shots = 1000

combined_circs = {}
count = {}

most_similar, most_similar_score = "", -1.0

for other_key in inverse_circs:
    if other_key == key:
        continue
        
    combined_circs[other_key] = circs[key] + inverse_circs[other_key]   #combined circuits to look for similar codes
    job = execute(combined_circs[other_key], backend=backend,shots=shots)
    st = job.result().get_counts(combined_circs[other_key])
    if "0"*n in st:
        sim_score = st["0"*n]/shots
    else:
        sim_score = 0.0
    
    print("Similarity score of",key,"and",other_key,"is",sim_score)
    if most_similar_score < sim_score:
        most_similar, most_similar_score = other_key, sim_score

print("[ANSWER]", key,"is most similar to", most_similar)


Similarity score of PROTOZOAN and YEAST is 0.857
Similarity score of PROTOZOAN and BACTERIAL is 0.97
[ANSWER] PROTOZOAN is most similar to BACTERIAL


We observe that the test can be used to determine which code is closer: bacterial's is closer to protozoan's than yeast's.

There are many other genetic codes listed at bioinformatics.org which can be used as input strings. In general, DNA has four nucleotides: "A", "C", "G", and "T". Thus, instead of one qubit like in this notebook, two qubits are required to encode the nucleotides. However, the asymptotic amount of quantum bits for encoding the whole sequence of length N is still in the order of logN, which is exponentially small.



In [7]:
print(qc_yeast)

                                                                            »
q0_0: |0>───────────────────────────────────────────────────────────────────»
                                                                            »
q0_1: |0>───────────────────────────────────────────────────────────────────»
                                                                            »
q0_2: |0>───────────────────────────────────────────────────────────────────»
                                                                            »
q0_3: |0>───────────────────────────────────────────────────────────────────»
                                     ┌───┐┌───┐┌───┐┌───┐┌────────────┐┌───┐»
q0_4: |0>────────────────────────────┤ X ├┤ X ├┤ X ├┤ X ├┤ Ry(1.5708) ├┤ X ├»
                       ┌────────────┐└─┬─┘└─┬─┘└─┬─┘└─┬─┘└────────────┘└─┬─┘»
q0_5: |0>──────────────┤ Ry(1.5708) ├──┼────■────┼────■──────────────────┼──»
         ┌────────────┐└────────────┘  │         │              

In [8]:
print(inverse_qc_yeast)

         ┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
q0_0: |0>┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├┤ X ├»
         └─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘└─┬─┘»
q0_1: |0>──■────┼────■────┼────■────┼────■────┼────■────┼────■────┼────■────┼──»
                │         │         │         │         │         │         │  »
q0_2: |0>───────■─────────┼─────────■─────────┼─────────■─────────┼─────────■──»
                          │                   │                   │            »
q0_3: |0>─────────────────■───────────────────┼───────────────────■────────────»
                                              │                                »
q0_4: |0>─────────────────────────────────────■────────────────────────────────»
                                                                               »
q0_5: |0>──────────────────────────────────────────────────────────────────────»
                            